In [1]:
from imutils.perspective import four_point_transform

In [2]:
from imutils import contours

In [3]:
import numpy as np

In [4]:
import argparse

In [5]:
import imutils

In [6]:
import cv2

In [7]:
ANSWER_KEY= {0: 1,1: 4,2: 0,3: 3,4: 1}

In [8]:
image = cv2.imread("omr.png")

**code to show image**

In [9]:
cv2.imshow('Original',image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [10]:
gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

**code to see gray scale image**

In [11]:
cv2.imshow('Grayscale',gray)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [12]:
blurred = cv2.GaussianBlur(gray,(5,5),0)

In [13]:
edged = cv2.Canny(blurred,75,200)

**code to see edges in image**

In [14]:
cv2.imshow('Canny Edge Detection',edged)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [15]:
cnts = cv2.findContours(edged.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
docCnt = None

In [16]:
if len(cnts)>0:
    cnts = sorted(cnts,key=cv2.contourArea,reverse=True)
    for c in cnts:
        peri = cv2.arcLength(c,True)
        approx=cv2.approxPolyDP(c,0.02*peri,True)
        if len(approx) == 4:
            docCnt=approx
            break

**code to see drawn contour on image**

In [17]:
cv2.drawContours(image,docCnt,-1,(0,255,0),3)
cv2.imshow('docCnt',image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [18]:
paper = four_point_transform(image,docCnt.reshape(4,2))

In [19]:
warped= four_point_transform(gray,docCnt.reshape(4,2))

**code to see paper and warped images**

In [20]:
cv2.imshow('birds-eye',paper)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [21]:
cv2.imshow('birds-eye',warped)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [22]:
(T,threshInv)=cv2.threshold(warped,0,255,cv2.THRESH_BINARY_INV|cv2.THRESH_OTSU)

**code to see thresholded image**

In [23]:
cv2.imshow("Otsu's thresholding",threshInv)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [24]:
cnts = cv2.findContours(threshInv.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
questionCnts = []

In [25]:
for c in cnts:
    (x,y,w,h) = cv2.boundingRect(c)
    ar = w / float(h)
    if w>=20 and h>=20 and ar>=0.9 and ar<=1.1:
        questionCnts.append(c)

**code to print all question bubbles in paper**

In [26]:
cv2.drawContours(paper,questionCnts,-1,(0,255,0),3)
cv2.imshow('Questions',paper)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [27]:
len(questionCnts)

25

In [28]:
area=cv2.contourArea(cnts[50])

In [29]:
area

905.0

In [30]:
questionCnts = contours.sort_contours(questionCnts,method="top-to-bottom")[0]

In [31]:
correct =0

In [32]:
for (q,i) in enumerate(np.arange(0,len(questionCnts),5)):
    cnts=contours.sort_contours(questionCnts[i: i+5])[0]
    bubbled=None
    for(j,c) in enumerate(cnts):
        mask=np.zeros(threshInv.shape,dtype="uint8")
        cv2.drawContours(mask,[c],-1,255,-1)
        
        mask=cv2.bitwise_and(threshInv,threshInv,mask=mask)
        total=cv2.countNonZero(mask)
        if bubbled is None or total>bubbled[0]:
            bubbled=(total,j)
            
    color=(0,0,255)
    k=ANSWER_KEY[q]
    
    if k==bubbled[1]:
        color=(0,255,0)
        correct+=1
        
    cv2.drawContours(paper,[cnts[k]],-1,color,3)

In [35]:
score = (correct / 5.0) * 100
print("[INFO] score: {:.2f}%".format(score))
cv2.putText(paper, "{:.2f}%".format(score), (10, 30),
    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
cv2.imshow("Original", image)
cv2.imshow("Exam", paper)
cv2.waitKey(0)

[INFO] score: 100.00%


-1